In [1]:
import numpy as np

# Implementación de la red neuronal con retropropagación

In [43]:
class NetNode(object):

  def __init__(self):
    self.inputs = []
    self.weights = []
    self.value = None

In [85]:
class Network(object):

  def __init__(self, layers):
    self.net = [[NetNode() for _ in range(size)] for size in layers]
    sizes = len(layers)
    for layer in range(1, sizes):
        for node in self.net[layer]:
            for unit in self.net[layer - 1]:
                node.inputs.append(unit)
                node.weights.append(0)

  def accuracy(self, examples):
    correct = 0
    for x_test, y_test in examples:
        prediction = self.predict(x_test)
        if (y_test[prediction] == 1):
            correct += 1
    return correct / len(examples)

  def backpropagation(self, eta, examples, epochs):
    inputs = self.net[0]
    outputs = self.net[-1]
    layer_size = len(self.net)
    for layer in self.net[1:]:
        for node in layer:
            node.weights = [np.random.uniform() for _ in range(len(node.weights))]
    for epoch in range(epochs):
        for x_train, y_train in examples:
            for value, node in zip(x_train, inputs):
                node.value = value
            for layer in self.net[1:]:
                for node in layer:
                    in_val = [n.value for n in node.inputs]
                    unit_value = np.dot(in_val, node.weights)
                    node.value = self.relu(unit_value)
            delta = [[] for _ in range(layer_size)]
            err = [y_train[i] - outputs[i].value for i in range(len(outputs))]
            delta[-1] = [self.relu_prime(outputs[i].value) * err[i] for i in range(len(outputs))]
            hidden_layers = layer_size - 2
            for i in range(hidden_layers, 0, -1):
                layer = self.net[i]
                n_layers = len(layer)
                w = [[node.weights[l] for node in self.net[i + 1]] for l in range(n_layers)]
                delta[i] = [self.relu_prime(layer[j].value) * np.dot(w[j], delta[i + 1]) for j in range(n_layers)]
            for i in range(1, layer_size):
                layer = self.net[i]
                in_val = [node.value for node in self.net[i - 1]]
                n_layers = len(self.net[i])
                for j in range(n_layers):
                    layer[j].weights = np.add(layer[j].weights, np.multiply(eta * delta[i][j], in_val))
        print(f"epoch {epoch}/{epochs} | total error={np.sum(err)/len(examples)}")
  
  def predict(self, input_data):
      inputs = self.net[0]
      for v, n in zip(input_data, inputs):
          n.value = v
      for layer in self.net[1:]:
          for node in layer:
              in_val = [n.value for n in node.inputs]
              unit_value = np.dot(in_val, node.weights)
              node.value = self.relu(unit_value)
      outputs = self.net[-1]
      return outputs.index(max(outputs, key=lambda node: node.value))

  def relu(self, z):
      return max(0, z)

  def relu_prime(self, z):
      return 1 if z > 0 else 0

  def sigmoide(self, z):
      pass

  def sigmoide_prime(self, z):
      pass
    
  def set_weights(self, pesos):
      i = 0
      for layer in self.net:
            for unit in layer:
              unit.weights = pesos[i]
              i+=1
  def clean_weights(self, pesos):
      i = 0
      for layer in self.net:
            for unit in layer:
              unit.weights = pesos[i]
            i+=1

  def weights(self):
      pesos = []
      for layer in self.net:
            for unit in layer:
              pesos.append(unit.weights)
      return pesos

  def weights2(self):
      pesos = []
      i = 1
      for layer in self.net:
            print ('Capa :',i)
            for unit in layer:
              print(unit.weights)
              pesos.append(unit.weights)
            i+=1;
      

# Usando la red neuronal con un dataset

In [45]:
from sklearn import datasets
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

In [46]:
iris_X, iris_y = datasets.load_iris(return_X_y=True)

In [47]:
iris_x_normalized = normalize(iris_X, axis=0)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(iris_x_normalized, iris_y, test_size=0.2, shuffle=True)

In [49]:
y_train = np_utils.to_categorical(y_train, num_classes=3)
y_test = np_utils.to_categorical(y_test, num_classes=3)

In [50]:
examples = []
for i in range(len(X_train)):
    examples.append([X_train[i], y_train[i]])

In [86]:
red = Network([3, 4, 2])
red.backpropagation(0.1, examples, 500)

epoch 0/500 | total error=0.01633009062322393
epoch 1/500 | total error=0.015724775952831817
epoch 2/500 | total error=0.01529812555272073
epoch 3/500 | total error=0.014998166368933231
epoch 4/500 | total error=0.014786059639488229
epoch 5/500 | total error=0.014633976258582464
epoch 6/500 | total error=0.014522514871753182
epoch 7/500 | total error=0.014438390783518372
epoch 8/500 | total error=0.014372614978252308
epoch 9/500 | total error=0.014319153054510484
epoch 10/500 | total error=0.014273975855746005
epoch 11/500 | total error=0.014234403515346756
epoch 12/500 | total error=0.014198659132636886
epoch 13/500 | total error=0.014165568062920837
epoch 14/500 | total error=0.014134356606277234
epoch 15/500 | total error=0.014104517802555084
epoch 16/500 | total error=0.014075722211151716
epoch 17/500 | total error=0.01404775870845266
epoch 18/500 | total error=0.014020495255896149
epoch 19/500 | total error=0.013993852928273721
epoch 20/500 | total error=0.013967788734709392
epoch

In [52]:
#precisión alcanzada con los datos de entrenamiento
accuracy = red.accuracy(examples)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6583333333333333


In [53]:
#precisión alcanzada con los datos de prueba
examples = []
for i in range(len(X_test)):
    examples.append([X_test[i], y_test[i]])
accuracy = red.accuracy(examples)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7


In [ ]:
#probando con un dato
prediction = red.predict(X_test[2])
print(f"Desired output: {y_test[2]}")
print(f"Index of output: {prediction}")

In [87]:
result = red.weights()
print(result)

[[], [], [], array([ 1.69531918,  0.71819461, -1.27107156]), array([1.45200892, 0.50560968, 0.74707091]), array([ 1.66367156,  2.02245048, -2.73600309]), array([ 0.92890446,  3.36763433, -4.7012687 ]), array([ 0.13489675, -1.43064707,  2.07612749,  3.54361178]), array([ 1.99581202,  0.79423364,  3.05432376, -4.60509962])]


In [91]:
red.weights2()

Capa : 1
[]
[]
[]
Capa : 2
[ 1.69531918  0.71819461 -1.27107156]
[1.45200892 0.50560968 0.74707091]
[ 1.66367156  2.02245048 -2.73600309]
[ 0.92890446  3.36763433 -4.7012687 ]
Capa : 3
[ 0.13489675 -1.43064707  2.07612749  3.54361178]
[ 1.99581202  0.79423364  3.05432376 -4.60509962]


In [ ]:
print(result[9])

In [90]:
red.set_weights(result)

In [88]:
red.clean_weights(result)

# Modificación de la implementación de la red neuronal

In [ ]:
# agrega el método weigths() a la clase Network, de tal forma que permita obtener los pesos de las neuronas
# agrega el método set_weights() a la clase Network, de tal forma que permite definir los pesos de las neuronas

In [ ]:
# agrega los métodos sigmoide() y sigmoide_prime() a la clase Network
# modifica la clase Network, para que se pueda decidir qué función de activación utilizar: relu() o sigmoide()

In [ ]:
# los métodos predict() y accuracy() de la clase Network están implementados para resolver problemas de clasificación
# modifícalos de tal manera que también se puedan utilizar con problemas de regresión

In [ ]:
# modifica el método backpropagation() de tal manera que devuelva como resultado el array de valores de los nodos durante las épocas de entrenamiento

In [ ]:
# una vez implementados los cambios, entrena la red neuronal del ejemplo de los apuntes
examples = []
examples.append([[0.5, 0.67, 0.5], [0.25, 0.6]])

In [ ]:
# ejecuta la red neuronal para los datos de ejemplo de los apuntes
# comprueba los valores de los nodos y de los pesos
# los valores de los nodos tienen que ser los mismos que los de los apuntes
# los valores de los pesos son ligeramente diferentes, ¿por qué?

net = Network([3, 4, 2])

net.set_weights([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
# o
net.set_weights([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4], [0.5, 0.5, 0.5, 0.5], [0.6, 0.6, 0.6, 0.6]])

valores_nodos = net.backpropagation(0.9, examples, 1)

print(valores_nodos)
net.weights()